In [1]:
import pandas as pd
import numpy as np
import random
import pickle
import math 

In [2]:
adj_matrix = np.load("init_flight_capacity.npy")

with open("terminal_name.txt") as f:
    airport_id = f.readlines()
airport_id = [i.replace("\n", "") for i in airport_id]

with open('city_id2airports.pickle', 'rb') as handle:
    cityid2airportid = pickle.load(handle)

with open('cityid2population.pickle', 'rb') as handle:
    cityid2population = pickle.load(handle)
    
with open('cityid_passengers_from_to.pickle', 'rb') as handle:
    cityid_outprob = pickle.load(handle)
flights_data = pd.read_csv("flights.csv")
# flights_data
# adj_matrix.shape

In [3]:
cityid_outprob

{0: {'HND_SHA': 0.005299923542086606,
  'HND_TAK': 0.0222423020782651,
  'NRT_BOS': 0.005299923542086606,
  'HND_AXT': 0.012059498158059359,
  'NRT_LAX': 0.012528671717522763,
  'HND_OKJ': 0.010982136651143393,
  'NRT_AMS': 0.005299923542086606,
  'NRT_PER': 0.00199833182734413,
  'NRT_XMN': 0.001598665461875304,
  'HND_IZO': 0.015743379439772016,
  'NRT_SIN': 0.012528671717522763,
  'NRT_KHH': 0.0037533884757072356,
  'HND_KCZ': 0.010982136651143393,
  'HND_TKS': 0.026360603322443873,
  'HND_HIJ': 0.02837631194828665,
  'HND_AKJ': 0.018488913602557864,
  'HND_NGS': 0.018488913602557864,
  'NRT_JFK': 0.013918815597414333,
  'NRT_FRA': 0.006707444220476819,
  'NRT_SGN': 0.003214707722249253,
  'HND_ISG': 0.005074025161604226,
  'HND_OBO': 0.016855494543685273,
  'NRT_HAN': 0.001598665461875304,
  'NRT_KUL': 0.0037533884757072356,
  'HND_ASJ': 0.002988809341766873,
  'NRT_HEL': 0.0030061861402655176,
  'HND_GMP': 0.014457496350872316,
  'NRT_SYD': 0.013049975672482101,
  'NRT_ORD': 0.007

In [4]:
# fix nan
# cityid2population_e[3442] = 8604203.0
# cityid2population_e[86] = 7398000.0
# cityid2population_e[29867] = 593754.0


In [5]:
# cityid2airportid = {2451:["AAR", "AAL"],719:["ABI"] }       
# cityid2population = {2451: 237551.0, 719: 693494.0}
# cityid_outprob = {2451: {"AAR_PER": 0.4, "AAR_ABI": 0.3, "AAL_ABI":0.3}, 719: {"ABI_AAR": 1.0}}

beta_city = [0.779 for i in range(len(cityid2airportid))]
gamma_city = [0.03 for i in range(len(cityid2airportid))]
beta_flight = np.load("beta.npy") # infection rate
gamma_flight = np.copy(beta_flight)
gamma_flight[gamma_flight != 0] = 0.03 # recovery rate


In [6]:
def ppl_same_city_in_plane(airports, probabilities, infection_num_out):
    # Use the random.choices function to fetch 15 elements
#     print(probabilities, airports)
    fetch = random.choices(airports, weights=probabilities, k=int(infection_num_out))

    # Count the number of times each element appears in the fetch
    infection_num_out_list=[]
    for item in airports:
        infection_num_out_list.append(fetch.count(item))
        
    return infection_num_out_list

def update_infection(arrive_infection_matrix):
    update_dictionary = dict(zip(airport_id, np.sum(arrive_infection_matrix, axis=0)))
    return update_dictionary

def get_infection_input(infection_dictionary):   
    city_infection_input = dict.fromkeys(cityid2airportid.keys(), 0)
    for k, v in cityid2airportid.items():
        infection_num = []
        for i in v:
            infection_num.append(infection_dictionary[i])
        city_infection_input[k] = sum(infection_num)
    return city_infection_input

def city_infection(city_infection_input, city_recovery):
    city_infection_num = np.array(list(city_infection_input.values()))
    city_population = np.array(list(cityid2population.values()))
    y_c = [(city_population - city_infection_num)/city_population, city_infection_num/city_population, city_recovery]
    
    s, i ,r = SIR(y_c, beta_city, gamma_city)
    
#     print("city_population: ",city_population)
    after_infection = np.ceil(i*city_population)

    return r, after_infection


def flight_infection(flight_adj, flight_infection_num):

    y_f = [np.nan_to_num((flight_adj-flight_infection_num)/flight_adj,nan = 1),\
           np.nan_to_num(flight_infection_num/flight_adj, nan = 0),\
           np.zeros(len(flight_adj))]
    
    
    s, i ,r = SIR(y_f, beta_flight, gamma_flight)
    arrive_infection = np.ceil(i*flight_adj)
    
    infection_ppl_flight = update_infection(arrive_infection)
    
    return infection_ppl_flight

def get_on_plane(flight_infection_matrix, infection_ppl_in_airport):
    for (k, v), num_of_people in zip(cityid_outprob.items(), infection_ppl_in_airport):
#         print((k, v), num_of_people)
        people_on_plane = ppl_same_city_in_plane(list(v.keys()), list(v.values()), num_of_people)
        flight_infection_matrix = update_flight_infection(flight_infection_matrix, list(v.keys()), people_on_plane)
    return flight_infection_matrix
    
def fly_infection_ppl(city_infection_ppl):
#     return np.nan_to_num(np.ceil(city_infection_ppl/list(cityid2population.values())),nan = 1)
    return np.nan_to_num(np.ceil(city_infection_ppl/1),nan = 1)
    

def SIR(y, beta, gamma):
    S, I, R = y 
    d0 = S - beta*S*I # derivative of S(t)
    d1 = I + beta*S*I - gamma*I # derivative of I(t)
    d2 = R+ gamma*I # derivative of R(t)
    
    return [d0, d1, d2]

def update_flight_infection(flight_infection, flight_com, ppl_num_list):
    flight_com = [i.split("_") for i in flight_com]
    for (flight_from, flight_to), infection_num in zip(flight_com, ppl_num_list):
#         print(flight_com, ppl_num_list)
        flight_from_id = airport_id.index(flight_from)
        flight_to_id = airport_id.index(flight_to)
        flight_infection[flight_from_id][flight_to_id] += infection_num
#     print(np.isnan(flight_infection).any())
    return flight_infection
    

def loop_infection(n_iter, flight_adj):
    
    flight_infection_num = np.zeros((len(flight_adj), len(flight_adj)))
    flight_infection_num[0][1] = 5
    
    city_recovery = [0.03 for i in range(len(cityid2airportid))]
    infection_ppl_list = [1]
#     print(flight_infection_num)
    
    for _ in range(n_iter):
        infection_ppl_flight = flight_infection(flight_adj, flight_infection_num)
        infection_ppl_city = get_infection_input(infection_ppl_flight)
#         print(infection_ppl_city)
        city_recovery, infection_ppl = city_infection(infection_ppl_city, city_recovery)
        flight_infection_num = get_on_plane(np.zeros((len(flight_adj), len(flight_adj))), fly_infection_ppl(infection_ppl))
#         print(flight_infection_num[2]) 
        infection_ppl_list.append(np.sum(infection_ppl))
    
    return flight_infection_num, infection_ppl_list
        


# adj_matrix = np.zeros((427,427))
# adj_matrix[0][2]=9
# adj_matrix
# beta_flight = np.zeros((427,427))
# beta_flight[0][2]=0.18

# gamma_flight = np.copy(beta_flight)
# gamma_flight[gamma_flight != 0] = 0.03 # recovery rate

matrix, infection_list = loop_infection(10, adj_matrix)
print(sum(infection_list), infection_list)
matrix

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

/opt/anaconda3/envs/ns/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: invalid value encountered in true_divide
/opt/anaconda3/envs/ns/lib/python3.7/site-packages/ipykernel_launcher.py:42: RuntimeWarning: invalid value encountered in true_divide
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[['HKG', 'BKK'], ['HKG', 'HEL'], ['HKG', 'CNS'], ['HKG', 'AMS'], ['HKG', 'BNE'], ['HKG', 'XMN'], ['HKG', 'FUK'], ['HKG', 'SHE'], ['HKG', 'BLR'], ['HKG', 'NRT'], ['HKG', 'TAO'], ['HKG', 'HGH'], ['HKG', 'KHH'], ['HKG', 'KWL'], ['HKG', 'FOC'], ['HKG', 'CKG'], ['HKG', 'FCO'], ['HKG', 'ICN'], ['HKG', 'DEL'], ['HKG', 'JNB'], ['HKG', 'SIN'], ['HKG', 'CMB'], ['HKG', 'KUL'], ['HKG', 'ADL'], ['HKG', 'CGK'], ['HKG', 'JFK'], ['HKG', 'PEK'], ['HKG', 'HAN'], ['HKG', 'SYX'], ['HKG', 'CTU'], ['HKG', 'DLC'], ['HKG', 'SYD'], ['HKG', 'SUB'], ['HKG', 'WUH'], ['HKG', 'AMM'], ['HKG', 'LHR'], ['HKG', 'RUH'], ['HKG', 'BKI'], ['HKG', 'PER'], ['HKG', 'CTS'], ['HKG', 'NKG'], ['HKG', 'HKT'], ['HKG', 'MEL'], ['HKG', 'MNL'], ['HKG', 'LAX'], ['HKG', 'PVG'], ['HKG', 'KHI'], ['HKG', 'SFO'], ['HKG', 'MAA'], ['HKG', 'CDG'], ['HKG', 'DPS'], ['HKG', 'AKL'], ['HKG', 'SGN'], ['HKG', 'YYZ'], ['HKG', 'DXB'], ['HKG', 'NGO'], ['HKG', 'YVR'], ['HKG', 'HND'], ['HKG', 'TPE'], ['HKG', 'KTM'], ['HKG', 'FRA'], ['HKG', 'DAC']] [82, 11

[['MAD', 'CCS'], ['MAD', 'ALC'], ['MAD', 'PNA'], ['MAD', 'MIA'], ['MAD', 'VLC'], ['MAD', 'BJZ'], ['MAD', 'XRY'], ['MAD', 'SDR'], ['MAD', 'EZE'], ['MAD', 'BUD'], ['MAD', 'SJO'], ['MAD', 'ORD'], ['MAD', 'MVD'], ['MAD', 'BCN'], ['MAD', 'BOS'], ['MAD', 'LIN'], ['MAD', 'BOD'], ['MAD', 'TLS'], ['MAD', 'ZAZ'], ['MAD', 'ATH'], ['MAD', 'MEX'], ['MAD', 'ARN'], ['MAD', 'PSA'], ['MAD', 'GYE'], ['MAD', 'FRA'], ['MAD', 'DUB'], ['MAD', 'FCO'], ['MAD', 'NCE'], ['MAD', 'LYS'], ['MAD', 'BLQ'], ['MAD', 'NAP'], ['MAD', 'LIS'], ['MAD', 'OTP'], ['MAD', 'DKR'], ['MAD', 'JNB'], ['MAD', 'OPO'], ['MAD', 'LIM'], ['MAD', 'HEL'], ['MAD', 'IST'], ['MAD', 'VCE'], ['MAD', 'LIL'], ['MAD', 'SSG'], ['MAD', 'AMS'], ['MAD', 'VIE'], ['MAD', 'GVA'], ['MAD', 'GUA'], ['MAD', 'SCQ'], ['MAD', 'PRG'], ['MAD', 'MRS'], ['MAD', 'MPL'], ['MAD', 'CMN'], ['MAD', 'TFS'], ['MAD', 'PTY'], ['MAD', 'SDQ'], ['MAD', 'MJV'], ['MAD', 'BOG'], ['MAD', 'IBZ'], ['MAD', 'WAW'], ['MAD', 'VGO'], ['MAD', 'LEI'], ['MAD', 'IAD'], ['MAD', 'CAI'], ['MAD',

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
SIR([np.array([0.99, 0.95]), np.array([0.1, 0.04]), np.array([0, 0.005])], np.array([0.779 for i in range(2)]), np.array([0.03 for i in range(2)]))

[array([0.912879, 0.920398]),
 array([0.174121, 0.068402]),
 array([0.003 , 0.0062])]

In [8]:
adj_matrix

array([[  0.,   9.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0., 106.],
       [  0.,   0.,   0., ...,   0.,  54.,   0.]])